# Example workflow

In [11]:
import rectanglepy as rectangle
from anndata import AnnData

## Load tutorial data

In [12]:
sc_counts, annotations, bulks  = rectangle.load_tutorial_data()

In [13]:
sc_counts.iloc[:, :10].head()

,NOC2L,HES4,ISG15,SDF4,B3GALT6,UBE2J2,INTS11,AURKAIP1,CCNL2,MRPL20
E2L4_GATGCTACAGGCACAA,0,11,7,0,2,1,2,5,0,0
L3_AACGTCACATCAGCAT,0,0,1,0,0,0,1,1,0,1
L3_GGAATGGGTCTAACGT,0,0,0,2,0,0,0,1,0,3
E2L7_GTCATTTAGGCCTTCG,2,1,1,2,0,0,0,1,0,0
E2L6_CCCTTAGCATTGACCA,0,0,0,1,0,0,0,1,2,1


In [14]:
annotations.head()

E2L4_GATGCTACAGGCACAA    Monocytes
L3_AACGTCACATCAGCAT      Monocytes
L3_GGAATGGGTCTAACGT      Monocytes
E2L7_GTCATTTAGGCCTTCG    Monocytes
E2L6_CCCTTAGCATTGACCA    Monocytes
Name: 0, dtype: object

In [15]:
bulks.iloc[:, :10].head()

,SNORA52,SNORA53,SNORA54,SNORA55,SNORA56,SNORA57,SNORA59B,SNORA5A,SNORA5B,SNORA5C
pbmc_1,2.02120,0.426412,0.0,0.585614,0.00000,2.060440,0.666746,1.84082,0.000000,2.53766
pbmc_10,1.15102,0.000000,0.0,1.185750,1.15618,0.325934,0.379692,2.01894,0.884945,1.70451


### Convert to AnnData object

In [16]:
sc_adata = AnnData(sc_counts, obs=annotations.to_frame(name="cell_type"))

# Single step Rectangle workflow

In [ ]:
estimations, signature_result = rectangle.rectangle(sc_adata, bulks)

In [18]:
estimations

,B cells,Monocytes,NK cells,Plasma cells,T cells CD4 conv,T cells CD8,Tregs,mDC,pDC,Unknown
pbmc_1,-1.653677e-18,0.105667,0.029340,0.008662,3.723346e-01,5.698229e-02,1.376688e-02,0.164630,0.248618,0.0
pbmc_10,1.641017e-17,0.242756,0.061698,0.001183,2.935494e-16,3.265707e-17,-3.973413e-17,0.278038,0.416326,0.0


signature_result is a [`RectangleSignatureResult`](../generated/rectanglepy.pp.RectangleSignatureResult.rst) object

## Even sampling and consensus

We recommend even subsampling of the sc data.
You can use the `sample_size` parameter to sample a fixed number of cells per cell type,  and run the consensus multiple times to get a more robust result.

In [ ]:
estimations, signature_result = rectangle.rectangle(sc_adata, bulks, consensus_runs=5, sample_size=1500)

# 2-step Rectangle workflow

Rectangle can also be run in two steps, first creating the signature and then deconvoluting the bulk data.

## Create Signature result

In [ ]:
signature_result = rectangle.pp.build_rectangle_signatures(sc_adata, bulks=bulks)

This creates a [`RectangleSignatureResult`](../generated/rectanglepy.pp.RectangleSignatureResult.rst) object.

## Deconvolute bulk data

In [ ]:
estimations = rectangle.tl.deconvolution(signature_result, bulks)

This returns a pandas DataFrame with the estimated cell type proportions.